In [440]:
#import libraries/dependencies
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
from sqlalchemy import create_engine
import pymysql

In [441]:
#import login credentials. This file is ignored via the gitignore file
from config import user, password

In [442]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

In [443]:
#create connection to MySQL Database
pymysql.install_as_MySQLdb()
engine = create_engine('mysql://' + user + ':' + password + '@imdb-project.cj1m13sfkxlg.us-east-2.rds.amazonaws.com:3306/baseball')

In [444]:
#use pandas for reading data from your MySQL server:
df = pd.read_sql('SELECT * FROM Dodgers', con=engine)

In [445]:
query= """
SELECT Game as Game_ID, `Number` as Opponent_Number,
REPLACE(LEFT(YYYYMMDD,10),'-','') as Date,
DAYOFWEEK(YYYYMMDD),
CASE WHEN `D/N` = 'D' THEN 1 ELSE 0 END as Day_or_Night, 
LENGTH(Streak) - LENGTH(REPLACE(Streak,'+','')) as Win_Streak,
Bobble_head,
precip,
Temp,
Attendance
FROM Dodgers_data as dodger
LEFT JOIN Opponent_Map as map on dodger.Opponent = map.Opponent
"""

In [446]:
df2 = pd.read_sql_query(query,engine)
df2['Attendance'] = df2['Attendance'].str.replace(',', '')
df2['Attendance'] = df2['Attendance'].astype(int)
df2.drop('Date', axis=1, inplace=True)
df2.head()


,Game_ID,Opponent_Number,DAYOFWEEK(YYYYMMDD),Day_or_Night,Win_Streak,Bobble_head,precip,Temp,Attendance
0,1,1,2,1,1,0,0.0,60.0,53701
1,2,1,3,0,0,0,0.0,62.0,42196
2,3,1,4,0,1,0,0.0,67.0,38373
3,4,1,5,1,2,0,0.0,66.0,36501
4,115,1,6,0,0,0,0.0,86.0,52898


In [447]:
target = df2["Attendance"]
target_names = ["Attendance"]

In [448]:
df3 = df2.drop("Attendance", axis=1)
feature_names = df3.columns
df3.head(5)

,Game_ID,Opponent_Number,DAYOFWEEK(YYYYMMDD),Day_or_Night,Win_Streak,Bobble_head,precip,Temp
0,1,1,2,1,1,0,0.0,60.0
1,2,1,3,0,0,0,0.0,62.0
2,3,1,4,0,1,0,0.0,67.0
3,4,1,5,1,2,0,0.0,66.0
4,115,1,6,0,0,0,0.0,86.0


# Predict  attendance for win streaks of 0-5

In [449]:
preds = []
for win_streak in range(5):
    pred = [1, 1, 2, 1 ,win_streak,1,1,1]
    preds.append(pred)
rf.predict(preds)

array([46969.55 , 47759.418, 47374.099, 47355.072, 47302.466])

# Plain Random Forest

In [450]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df3, target)

In [451]:
# Create a random forest classifier
rf = RandomForestRegressor(n_estimators=1000)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.08276024988925335

# Feature Importances

In [452]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([0.2797539 , 0.18642946, 0.12702979, 0.01745131, 0.07712088,
       0.05745075, 0.04602283, 0.20874108])

In [453]:
feature_names= "Game_ID", "Opponent_Number", "DAYOFWEEK(YYYYMMDD)", "Day_or_Night", "Win_Streak", "Bobble_head", "precip","Temp"

In [454]:
# We can sort the features by their importance
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.27975390083645446, 'Game_ID'),
 (0.20874108458672086, 'Temp'),
 (0.18642945748737097, 'Opponent_Number'),
 (0.12702978705112375, 'DAYOFWEEK(YYYYMMDD)'),
 (0.07712088058412911, 'Win_Streak'),
 (0.057450753352336625, 'Bobble_head'),
 (0.04602282973563948, 'precip'),
 (0.01745130636622453, 'Day_or_Night')]